In [1]:
!pip install bcrypt

In [ ]:
import bcrypt
import re
from typing import Tuple, Optional

class PasswordError(Exception):
    """Custom exception for password-related errors"""
    pass

def validate_password_strength(password: str) -> Tuple[bool, Optional[str]]:
    """
    Validate password strength against security requirements.
    
    Requirements:
    - Minimum 8 characters
    - At least one uppercase letter
    - At least one lowercase letter
    - At least one number
    - At least one special character
    
    Args:
        password: The password to validate
        
    Returns:
        Tuple[bool, Optional[str]]: (is_valid, error_message)
    """
    if len(password) < 8:
        return False, "Password must be at least 8 characters long"
    
    if not re.search(r"[A-Z]", password):
        return False, "Password must contain at least one uppercase letter"
    
    if not re.search(r"[a-z]", password):
        return False, "Password must contain at least one lowercase letter"
    
    if not re.search(r"\d", password):
        return False, "Password must contain at least one number"
    
    if not re.search(r"[!@#$%^&*(),.?\":{}|<>]", password):
        return False, "Password must contain at least one special character"
    
    return True, None

def hash_password(password: str) -> bytes:
    """
    Hash a password using bcrypt with salt.
    
    Args:
        password: The plain text password to hash
        
    Returns:
        bytes: The hashed password with salt
        
    Raises:
        PasswordError: If password doesn't meet strength requirements
    """
    # First validate password strength
    is_valid, error_message = validate_password_strength(password)
    if not is_valid:
        raise PasswordError(error_message)
    
    # Generate a salt and hash the password
    # WorkFactor of 12 is considered secure as of 2024 while still being reasonably fast
    salt = bcrypt.gensalt(rounds=12)
    hashed = bcrypt.hashpw(password.encode('utf-8'), salt)
    return hashed

def verify_password(password: str, hashed_password: bytes) -> bool:
    """
    Verify a password against a hashed password.
    
    Args:
        password: The plain text password to verify
        hashed_password: The hashed password to check against
        
    Returns:
        bool: True if password matches, False otherwise
    """
    try:
        return bcrypt.checkpw(password.encode('utf-8'), hashed_password)
    except Exception:
        return False

In [12]:
# Example usage:
def example_usage():
    try:
        # Hashing a password
        password = "MySecure123!@#"
        hashed_password = hash_password(password)
        print(f"\nHashed password: {password} --> {hashed_password}")
        
        # Verifying a password
        is_valid = verify_password(password, hashed_password)
        print(f"...... Result: {is_valid}")
        
        # Testing invalid password
        password = "WrongPassword123!@#"
        print(f"\nHashed password: {password} --> {hashed_password}")
        is_valid = verify_password(password, hashed_password)
        print(f"Invalid password verification (should be False): {is_valid}")
        
        # Testing weak password
        pwd_list = ["weak", 'H7_Super_007@gmail.com', 'H7_User_131@gmail.com', 'H7_User_212@gmail.com']    
        for password in pwd_list:
            try:
                hashed_password = hash_password(password)
                print(f"\nHashed password: {password} --> {hashed_password}")
                is_valid = verify_password(password, hashed_password)
                print(f"...... Result: {is_valid}")
            except PasswordError as e:
                print(f"Weak password rejected: {str(e)}")
            
    except Exception as e:
        print(f"Error: {str(e)}")

if __name__ == "__main__":
    example_usage()


Hashed password: MySecure123!@# --> b'$2b$12$/ED5UWfvfWW2KyrB41x71.JL/fdw0Mcgzdv8e161upQGk0EGDOaz6'
...... Result: True

Hashed password: WrongPassword123!@# --> b'$2b$12$/ED5UWfvfWW2KyrB41x71.JL/fdw0Mcgzdv8e161upQGk0EGDOaz6'
Invalid password verification (should be False): False
Weak password rejected: Password must be at least 8 characters long

Hashed password: H7_Super_007@gmail.com --> b'$2b$12$B1wBS0lDL4eN7VwI97FSF.tvEnJnofd7QPeaZ1UNWM0V0iCyC.RKy'
...... Result: True

Hashed password: H7_User_131@gmail.com --> b'$2b$12$kz9iWqf2jgW/HOlZOC2jsuAXCdxjY9g5oQQC5a8NKodurQ1s2neBK'
...... Result: True

Hashed password: H7_User_212@gmail.com --> b'$2b$12$b0DpatShSnjdeam8vhVyu.ay59zaXdaXZXghl1F99/JyA1R.vVx7y'
...... Result: True


In [11]:
s = """ 
H7_Super_007@gmail.com
H7_User_131@gmail.com
H7_User_212@gmail.com

"""
[i for i in s.split()]

['H7_Super_007@gmail.com', 'H7_User_131@gmail.com', 'H7_User_212@gmail.com']

In [10]:
print("\n".join(["weak", 'H7_Super_007@gmail.com', 'H7_Usr_131@gmail.com', 'H7_Usr_214@gmail.com'] ))

weak
H7_Super_007@gmail.com
H7_Usr_131@gmail.com
H7_Usr_214@gmail.com
